In [ ]:
import pandas as pd
import re
#valtest_df1 = pd.read_csv('valtest_first_round1.csv')
#valtest_df2 = pd.read_csv('valtest_set_extra.csv')
#first_20 = pd.read_csv('sample_20.csv')

In [ ]:
#first_20 = first_20.drop(index =[3,6,7,8,16,17,18])


In [ ]:
#first_20 = first_20[first_20['report_type'].isin(['Voortgangsverslag', 'MDO-verslag'])]

In [ ]:
#print(len(first_20))

In [ ]:
valtest = pd.read_csv('second_round_104_files.csv')

In [ ]:
#valtest = pd.concat([valtest_df1,valtest_df2])
#valtest = valtest_df1

In [ ]:
valset = valtest.loc[valtest['set'] == 'val']
testset = valtest.loc[valtest['set'] == 'test']

In [ ]:
print(len(valset))
print(len(testset))
print(valset['pseudo_id'].unique())
print(testset['pseudo_id'].unique())

In [ ]:
print(valset['report_type'].unique())

In [ ]:
print(testset['report_type'].unique())

In [ ]:
#print(valtest_df1['report_type'].unique())
#print(valtest_df2['report_type'].unique())

In [ ]:
data = valtest
#The text of each note shall not contain any additional space/period. Therefore I replace most common combination of codes for line termination (\r and \n)
data['pseudonomised_text']=data['pseudonomised_text'].str.replace('\r\n\r\n\r\n' ,'. ')
data['pseudonomised_text']=data['pseudonomised_text'].str.replace('\r\n',': ')
data['pseudonomised_text']=data['pseudonomised_text'].str.replace('\r\n\r\n\r\n' ,'. ')

In [ ]:
#data = data[~data['note_nr'].isin(first_20['note_nr'])]

In [ ]:
print(data['report_type'].unique())

In [ ]:
print(len(data))

In [ ]:
import re

def find_performance_scores(text):
    """
    Finds performance scores in a given text and extracts surrounding context.

    Args:
        text: The text to search.

    Returns:
        A list of dictionaries containing the score type, system (for ECOG/WHO),
        and the original surrounding context (if a match is found).
    """

    scores = []

    # Define patterns to capture WHO and ECOG scores, including all variations from PS_List
    patterns = [
        r"(?P<before>.{0,50})(WHO(?:[-\s]?score[:\s-]*)?(?:\d+[-\s]?\d*)|WHO(?:-schaal)?[:\s-]*\d+|WHO-classificatie[:\s-]*\d+|ECOG[:\s-]?\d+[-\s]?\d*)(?P<after>.{0,50})",
        r"(?P<before>.{0,50})(performance\s+status\s*(?:ECOG|WHO)\s*\d+)(?P<after>.{0,50})",
        r"(?P<before>.{0,50})(Karnofsky[:\s-]*\d+)(?P<after>.{0,50})",
        r"(?P<before>.{0,50})(PS(?:\s*\.\s*|\s*|\.|\s*PS\s*)\d+)(?P<after>.{0,50})"
    ]

    for pattern in patterns:
        for match in re.finditer(pattern, text):
            score_type = "WHO" if "WHO" in match.group(2) else ("ECOG" if "ECOG" in match.group(2) else "Karnofsky" if "Karnofsky" in match.group(2) else "PS")
            score_system = match.group(2)
            context = match.group("before") + match.group(2) + match.group("after")
            scores.append({
                "type": score_type,
                "system": score_system,
                "context": context
            })

    return scores

def create_has_score_column(text):
  """
  Checks if there's a match in the text and returns the context if a match is found,
  otherwise returns an empty string.
  """
  scores = find_performance_scores(text)
  if scores:
    return scores[0]["context"]  # Assuming only the first match is needed
  else:
    return ""

# Add a new column named 'has_score' to store the context if a match is found
data['has_score'] = data['pseudonomised_text'].apply(create_has_score_column)

print(data.columns)

# Print the DataFrame to see the new 'has_score' column
print('length df:', len(data))


In [ ]:
print(data.columns)

In [ ]:

def extract_and_convert_scores(text):
  """
  Extracts scores from a string, converts them based on ranges, and removes duplicates.

  Args:
    text: The text to extract scores from.

  Returns:
    A list of unique converted scores found in the text.
  """
  scores = []
  for match in re.finditer(r"(\d+)", text):
    score_value = int(match.group())
#     converted_value = None
#     if (score_value == 100) or (score_value == 90) :
#       converted_value = 0
#     elif (score_value == 80) or (score_value == 70) :
#       converted_value = 1
#     elif (score_value == 60) or (score_value == 50) :
#       converted_value = 2
#     elif (score_value == 40) or (score_value == 30) :
#       converted_value = 3
#     elif (score_value == 20) or (score_value == 10) :
#       converted_value = 4   
#     else:  # Scores below 30 remain the same
#       converted_value = score_value
    converted_value = score_value
    # Karnofsky is here not converted, because it is used to censor the file.

    if converted_value is not None:
      scores.append(converted_value)
  return list(set(scores))  # Ensure unique elements

# Create a new column named 'PS' to store the extracted and converted scores
data['PS'] = data['has_score'].apply(extract_and_convert_scores)

# Print the DataFrame to see the new 'PS' column
print('length df:', len(data))





In [ ]:
print(data.columns)

In [ ]:
data['has_score'].nunique()

In [ ]:
print(data['has_score'].unique())

In [ ]:
print(data.columns)

# Censoring Performance Stati

In [ ]:
PS_df = data[data['has_score'] != '']

In [ ]:
len(PS_df)

In [ ]:
pd.options.display.max_colwidth = 5000
PS_df['pseudonomised_text']

In [ ]:

# Define a function to check if any string in the list is present in a sentence
#def contains_any(sentence, PS_List):
#  return any(word in sentence for word in PS_List)

# Filter rows where 'col1' contains at least one string from 'search_list'
#PS_df = PS_df[PS_df['pseudonomised_text'].apply(lambda x: contains_any(x, PS_List))]

#print(len(PS_df))

In [ ]:
PS_df['censored'] = PS_df['pseudonomised_text']

In [ ]:
print(data.columns)

In [ ]:
import pandas as pd

# List of strings to replace
replace_list = PS_List

# Replacement string
replacement_string = '[WHO PS gecensureerd]'

# Replace strings in 'col1' using a loop (for educational purposes)
def replace_strings_loop(df, replace_list, replacement_string, col_name='censored'):
  """
  Replaces strings in a DataFrame column using a loop.

  Args:
      df (pandas.DataFrame): The DataFrame containing the column to modify.
      replace_list (list): The list of strings to replace.
      replacement_string (str): The string to replace the elements in 'replace_list' with.
      col_name (str, optional): The name of the column to modify. Defaults to 'col1'.

  Returns:
      pandas.DataFrame: The DataFrame with the replacements applied.
  """
  for index, row in df.iterrows():
    # Make a copy of the row to avoid modifying the original DataFrame
    new_row = row.copy()
    for item in replace_list:
      new_row[col_name] = new_row[col_name].replace(item, replacement_string)
    df.loc[index] = new_row
  return df

# Replace strings using vectorized string methods (more efficient)
def replace_strings_vectorized(df, replace_list, replacement_string, col_name='censored'):
  """
  Replaces strings in a DataFrame column using vectorized string methods.

  Args:
      df (pandas.DataFrame): The DataFrame containing the column to modify.
      replace_list (list): The list of strings to replace.
      replacement_string (str): The string to replace the elements in 'replace_list' with.
      col_name (str, optional): The name of the column to modify. Defaults to 'col1'.

  Returns:
      pandas.DataFrame: The DataFrame with the replacements applied.
  """
  for item in replace_list:
    df[col_name] = df[col_name].str.replace(item, replacement_string)
  return df

# Example usage (loop method)
#replaced_df_loop = replace_strings_loop(PS_df.copy(), replace_list.copy(), replacement_string)

# Example usage (vectorized method - recommended for efficiency)
PS_df = replace_strings_vectorized(PS_df.copy(), replace_list.copy(), replacement_string)
print(PS_df)


In [ ]:
# import pandas as pd

# # List of strings to replace
# replace_list = PS_List + ['[WHO PS gecensureerd]-1']

# # Replacement string
# replacement_string = '[WHO PS gecensureerd]'

# # Replace strings in 'col1' using a loop (for educational purposes)
# def replace_strings_loop(df, replace_list, replacement_string, col_name='censored'):
#   """
#   Replaces strings in a DataFrame column using a loop.

#   Args:
#       df (pandas.DataFrame): The DataFrame containing the column to modify.
#       replace_list (list): The list of strings to replace.
#       replacement_string (str): The string to replace the elements in 'replace_list' with.
#       col_name (str, optional): The name of the column to modify. Defaults to 'col1'.

#   Returns:
#       pandas.DataFrame: The DataFrame with the replacements applied.
#   """
#   for index, row in df.iterrows():
#     # Make a copy of the row to avoid modifying the original DataFrame
#     new_row = row.copy()
#     for item in replace_list:
#       new_row[col_name] = new_row[col_name].replace(item, replacement_string)
#     df.loc[index] = new_row
#   return df

# # Replace strings using vectorized string methods (more efficient)
# def replace_strings_vectorized(df, replace_list, replacement_string, col_name='censored'):
#   """
#   Replaces strings in a DataFrame column using vectorized string methods.

#   Args:
#       df (pandas.DataFrame): The DataFrame containing the column to modify.
#       replace_list (list): The list of strings to replace.
#       replacement_string (str): The string to replace the elements in 'replace_list' with.
#       col_name (str, optional): The name of the column to modify. Defaults to 'col1'.

#   Returns:
#       pandas.DataFrame: The DataFrame with the replacements applied.
#   """
#   for item in replace_list:
#     df[col_name] = df[col_name].str.replace(item, replacement_string)
#   return df

# #  (loop method)
# #replaced_df_loop = replace_strings_loop(PS_df.copy(), replace_list.copy(), replacement_string)

# #  (vectorized method - recommended for efficiency)
# replaced_df_vec = replace_strings_vectorized(PS_df.copy(), replace_list.copy(), replacement_string)
# print(replaced_df_vec)

import pandas as pd
import re

# List of strings to replace
replace_list = PS_List + ['[WHO PS gecensureerd]-1']

# Replacement string
replacement_string = '[WHO PS gecensureerd]'

# Replace strings using regular expressions
def replace_strings_with_regex(df, replacement_string, col_name='censored'):
    """
    Replaces strings in a DataFrame column using regular expressions.

    Args:
        df (pandas.DataFrame): The DataFrame containing the column to modify.
        replacement_string (str): The string to replace the matched patterns with.
        col_name (str, optional): The name of the column to modify. Defaults to 'censored'.

    Returns:
        pandas.DataFrame: The DataFrame with the replacements applied.
    """
    patterns = [
        r"WHO(?:[-\s]?score[:\s-]*)?(?:\d+[-\s]?\d*)",
        r"WHO(?:-schaal)?[:\s-]*\d+",
        r"WHO-classificatie[:\s-]*\d+",
        r"ECOG[:\s-]?\d+[-\s]?\d*",
        r"performance\s+status\s*(?:ECOG|WHO)\s*\d+",
        r"Karnofsky[:\s-]*\d+",
        r"PS(?:\s*\.\s*|\s*|\.|\s*PS\s*)\d+"
    ]

    for pattern in patterns:
        df[col_name] = df[col_name].str.replace(pattern, replacement_string, regex=True)
    
    return df

replaced_df_vec = replace_strings_vectorized(PS_df.copy(), replace_list.copy(), replacement_string)
print(replaced_df_vec)


In [ ]:
# all rows with PS
PS_df = replaced_df_vec
# all rows without PS
no_PS = data[~data['note_nr'].isin(PS_df['note_nr'])]
no_PS['censored'] = no_PS['pseudonomised_text']

In [ ]:
print(len(PS_df))

In [ ]:
# sample for inter-annotator agreement
from sklearn.utils import shuffle
PS_df_sample = PS_df.sample(n=11)
no_PS_df_sample = no_PS.sample(n=29)
sample_20_a = pd.concat([PS_df_sample[:10], no_PS_df_sample[:10]])
sample_20_b = pd.concat([PS_df_sample[10:], no_PS_df_sample[10:]])
#sample_40 = pd.concat([PS_df_sample, no_PS_df_sample])
#sample_40 = shuffle(sample_40)

In [ ]:
print(len(sample_20_a))
print(len(sample_20_b))
print(sample_20_a['note_nr'].unique())
print(sample_20_b['note_nr'].unique())
print(sample_20_a['has_score'])
print(sample_20_b['has_score'])
print(sample_20_a['censored'])
print(sample_20_b['censored'])
sample_20_a = shuffle(sample_20_a)
sample_20_b = shuffle(sample_20_b)

In [ ]:
# shuffle dataset
first_round = pd.concat([PS_df, no_PS])
first_round = shuffle(first_round)

In [ ]:
import pandas as pd

# List of strings to replace
replace_list = PS_List

# Replacement string
replacement_string = '[WHO PS gecensureerd]'

# Replace strings in 'col1' using a loop (for educational purposes)
def replace_strings_loop(df, replace_list, replacement_string, col_name='censored'):
  """
  Replaces strings in a DataFrame column using a loop.

  Args:
      df (pandas.DataFrame): The DataFrame containing the column to modify.
      replace_list (list): The list of strings to replace.
      replacement_string (str): The string to replace the elements in 'replace_list' with.
      col_name (str, optional): The name of the column to modify. Defaults to 'col1'.

  Returns:
      pandas.DataFrame: The DataFrame with the replacements applied.
  """
  for index, row in df.iterrows():
    # Make a copy of the row to avoid modifying the original DataFrame
    new_row = row.copy()
    for item in replace_list:
      new_row[col_name] = new_row[col_name].replace(item, replacement_string)
    df.loc[index] = new_row
  return df

# Replace strings using vectorized string methods (more efficient)
def replace_strings_vectorized(df, replace_list, replacement_string, col_name='censored'):
  """
  Replaces strings in a DataFrame column using vectorized string methods.

  Args:
      df (pandas.DataFrame): The DataFrame containing the column to modify.
      replace_list (list): The list of strings to replace.
      replacement_string (str): The string to replace the elements in 'replace_list' with.
      col_name (str, optional): The name of the column to modify. Defaults to 'col1'.

  Returns:
      pandas.DataFrame: The DataFrame with the replacements applied.
  """
  for item in replace_list:
    df[col_name] = df[col_name].str.replace(item, replacement_string)
  return df

# Example usage (loop method)
#replaced_df_loop = replace_strings_loop(PS_df.copy(), replace_list.copy(), replacement_string)

# Example usage (vectorized method - recommended for efficiency)
first_round = replace_strings_vectorized(first_round.copy(), replace_list.copy(), replacement_string)

In [ ]:
#print(sample_40['censored'])

In [ ]:
first_round = first_round[~first_round['note_nr'].isin(sample_20_a['note_nr'])]
first_round = first_round[~first_round['note_nr'].isin(sample_20_b['note_nr'])]

In [ ]:
print(len(first_round))

In [ ]:
# split data for annotators
subset_size = int(len(first_round)/2)
data_ann1 = first_round.sample(n=subset_size,random_state =1)
data_ann2 = first_round.drop(data_ann1.index)

In [ ]:
# sample 20b is used for second inter-annotator agreement-moment
first_round_ann1 = pd.concat([data_ann1, sample_20_b], ignore_index=True)
first_round_ann2 = pd.concat([data_ann2, sample_20_b], ignore_index=True)

In [ ]:
print(len(first_round))
print(first_round['note_nr'].nunique())

In [ ]:
print(first_round[first_round['censored'].str.contains('WHO', case=False, na=False)])

In [ ]:
print(first_round_ann1['note_nr'].tail(30))
print(first_round_ann2['note_nr'].tail(30))

In [ ]:
print(first_round_ann1['censored'].tail(30))
print(first_round_ann2['censored'].tail(30))

In [ ]:


first_round_ann1.to_csv('first_round_ann1_allcol.csv')
first_round_ann2.to_csv('first_round_ann2_allcol.csv')

In [ ]:
first_round_ann1 = first_round_ann1[['note_nr', 'report_type', 'censored', 'set']]
first_round_ann2 = first_round_ann2[['note_nr', 'report_type', 'censored', 'set']]

In [ ]:
print(sample_20_a['censored'])

In [ ]:
#sample_20_a = sample_20_a[['note_nr', 'report_type', 'censored']]
#sample_20_a.to_csv('sample_20_iaa.csv')

In [ ]:
print()

In [ ]:
directory_path = r"C:\Users\Nynke (A193DBBA)\Desktop\\Second_round"
first_round_ann2 = pd.concat([first_round_ann2, first_round_ann1[:10]])


In [ ]:
for index, row in first_round_ann1.iterrows():
    note_nr = row['note_nr']
    row.to_csv(f'{directory_path}ann1_file_{note_nr}', sep='.')
    
for index, row in first_round_ann2.iterrows():
    note_nr = row['note_nr']
    row.to_csv(f'{directory_path}ann2_file_{note_nr}', sep='.')

In [ ]:
#for index, row in sample_20_a.iterrows():
#    row.to_csv(f'{dir_path}\iaa-file{index}', sep='.')

In [ ]:
print(PS_df['censored'])

In [ ]:
print(len(first_round))

In [ ]:
first_round_ann2 = pd.concat([first_round_ann2, first_round_ann1[:10]])

In [ ]:
print(len(first_round_ann1))
print(len(first_round_ann2))

In [ ]:
print(len(valtest))

In [ ]:
print(set(first_round_ann1['note_nr']).intersection(set(first_round_ann2['note_nr'])))

In [ ]:
print(first_round_ann1.columns)